# 1️⃣ Training an Adapter for a Transformer Model

In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using the _[Adapters](https://github.com/Adapter-Hub/adapters)_ library and Hugging Face's _Transformers_ library.

We train a **[bottleneck adapter](https://docs.adapterhub.ml/methods.html#bottleneck-adapters)** on top of a pre-trained model here. Most of the code is identical to a full fine-tuning setup using _Transformers_.

For training, we use the [movie review dataset by Pang and Lee (2005)](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains movie reviews from Rotten Tomatoes which are either classified as positive or negative. We download the dataset via Hugging Face's [_Datasets_](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [1]:
#from datasets import load_dataset

#dataset = load_dataset("rotten_tomatoes")
#dataset.num_rows

In [52]:
from datasets import load_dataset, load_metric
task = "sst2"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

/Users/mohamadh/PycharmProjects/adapter/.venv/lib/python3.9/site-packages/datasets/load.py:756: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [57]:
dataset["test"].features["label"]

ClassLabel(names=['negative', 'positive'], id=None)

In [60]:
dataset["validation"].features["label"]

ClassLabel(names=['negative', 'positive'], id=None)

"
Every dataset sample has an input text and a binary label:

In [3]:
dataset['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [62]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["sentence"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [67]:
dataset["validation"].features["labels"]

ClassLabel(names=['negative', 'positive'], id=None)

In [85]:
dataset["test"].features

{'sentence': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

Now we're ready to train our model...

## Training

We use a pre-trained RoBERTa model checkpoint from the Hugging Face Hub. We load it with [`AutoAdapterModel`](https://docs.adapterhub.ml/classes/models/auto.html), a class unique to `adapters`. In addition to regular _Transformers_ classes, this class comes with all sorts of adapter-specific functionality, allowing flexible management and configuration of multiple adapters and prediction heads. [Learn more](https://docs.adapterhub.ml/prediction_heads.html#adaptermodel-classes).

In [5]:
from transformers import DistilBertConfig
from adapters import AutoAdapterModel

config = DistilBertConfig.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
)
model = AutoAdapterModel.from_pretrained(
    "distilbert-base-uncased",
    config=config,
)

/Users/mohamadh/PycharmProjects/adapter/.venv/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/mohamadh/PycharmProjects/adapter/.venv/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and an adapter configuration. `"bn_seq"` denotes a [sequential bottleneck adapter](https://docs.adapterhub.ml/methods.html#bottleneck-adapters) configuration.
_Adapters_ supports a diverse range of different adapter configurations. For example, `config="lora"` can be passed for training a [LoRA](https://docs.adapterhub.ml/methods.html#lora) adapter or `config="prefix_tuning"` for a [prefix tuning](https://docs.adapterhub.ml/methods.html#prefix-tuning). You can find all currently supported configs [here](https://docs.adapterhub.ml/methods.html#prefix-tuning).

Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model, so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [6]:
# Add a new adapter
model.add_adapter("sst2", config="seq_bn")
# Alternatively, e.g.:
# model.add_adapter("rotten_tomatoes", config="lora")

# Add a matching classification head
model.add_classification_head(
    "sst2",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
model.train_adapter("sst2")

For training an adapter, we make use of the `AdapterTrainer` class built-in into _Adapters_. This class is largely identical to _Transformer_'s `Trainer`, with some helpful tweaks e.g. for checkpointing only adapter weights.

We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full fine-tuning.** Adapter training usually requires a few more training epochs than full fine-tuning.

In [7]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

/Users/mohamadh/PycharmProjects/adapter/.venv/lib/python3.9/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Start the training 🚀

In [8]:
import time
# Start the timer
start_time = time.time()

# Start training
train_out = trainer.train()

# Calculate and print the time taken
elapsed_time = time.time() - start_time
print(f"Training completed in {elapsed_time:.2f} seconds.")

Step,Training Loss
200,0.417900
400,0.331100
600,0.299400
800,0.307400
1000,0.288900
1200,0.282400
1400,0.269600
1600,0.272500
1800,0.264000
2000,0.257500


Training completed in 1456.68 seconds.


In [12]:
elapsed_time

1456.6841909885406

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [74]:
data = trainer.evaluate()
data

{'eval_loss': 0.3184843063354492,
 'eval_acc': 0.9025229357798165,
 'eval_runtime': 1.3112,
 'eval_samples_per_second': 665.018,
 'eval_steps_per_second': 21.354,
 'epoch': 6.0}

PredictionOutput(predictions=array([[ 1.4492902 , -1.5371469 ],
       [ 3.5633986 , -3.6648026 ],
       [-4.5918994 ,  4.5304594 ],
       ...,
       [-3.0706913 ,  3.1146638 ],
       [-1.7641047 ,  1.7901833 ],
       [-0.18581426,  0.173024  ]], dtype=float32), label_ids=array([0, 0, 1, ..., 1, 1, 0]), metrics={'test_loss': 0.14854145050048828, 'test_acc': 0.947334036140106, 'test_runtime': 143.4915, 'test_samples_per_second': 469.359, 'test_steps_per_second': 14.67})

In [76]:
pred_val = trainer.predict(dataset["validation"])
pred_val

PredictionOutput(predictions=array([[-5.34196138e+00,  5.31464863e+00],
       [ 3.49476993e-01, -4.30205405e-01],
       [-3.78820109e+00,  3.77217412e+00],
       [-3.79907417e+00,  3.70341229e+00],
       [ 3.07559299e+00, -3.02841663e+00],
       [-5.19867516e+00,  5.10429525e+00],
       [ 3.02087855e+00, -3.05559468e+00],
       [ 1.70224440e+00, -1.83606672e+00],
       [-3.47645092e+00,  3.32236171e+00],
       [ 3.43996215e+00, -3.48225141e+00],
       [-4.70404863e+00,  4.65137339e+00],
       [ 3.72734141e+00, -3.83677459e+00],
       [ 4.31954193e+00, -4.28847218e+00],
       [ 1.06383538e+00, -1.08941543e+00],
       [ 3.79060149e+00, -3.85970378e+00],
       [-5.46737194e+00,  5.38918018e+00],
       [-3.91926026e+00,  3.89356494e+00],
       [-3.89455462e+00,  3.74273014e+00],
       [ 3.79278517e+00, -3.98477817e+00],
       [ 3.56608748e+00, -3.73118353e+00],
       [ 1.81751800e+00, -2.03834414e+00],
       [ 1.54758310e+00, -1.80709028e+00],
       [-4.56965923e-01, 

In [77]:
pred_val

PredictionOutput(predictions=array([[-5.34196138e+00,  5.31464863e+00],
       [ 3.49476993e-01, -4.30205405e-01],
       [-3.78820109e+00,  3.77217412e+00],
       [-3.79907417e+00,  3.70341229e+00],
       [ 3.07559299e+00, -3.02841663e+00],
       [-5.19867516e+00,  5.10429525e+00],
       [ 3.02087855e+00, -3.05559468e+00],
       [ 1.70224440e+00, -1.83606672e+00],
       [-3.47645092e+00,  3.32236171e+00],
       [ 3.43996215e+00, -3.48225141e+00],
       [-4.70404863e+00,  4.65137339e+00],
       [ 3.72734141e+00, -3.83677459e+00],
       [ 4.31954193e+00, -4.28847218e+00],
       [ 1.06383538e+00, -1.08941543e+00],
       [ 3.79060149e+00, -3.85970378e+00],
       [-5.46737194e+00,  5.38918018e+00],
       [-3.91926026e+00,  3.89356494e+00],
       [-3.89455462e+00,  3.74273014e+00],
       [ 3.79278517e+00, -3.98477817e+00],
       [ 3.56608748e+00, -3.73118353e+00],
       [ 1.81751800e+00, -2.03834414e+00],
       [ 1.54758310e+00, -1.80709028e+00],
       [-4.56965923e-01, 

In [78]:
pred_test = trainer.predict(dataset["test"])
pred_test

PredictionOutput(predictions=array([[ 2.67072082e+00, -2.71354985e+00],
       [ 2.64563489e+00, -2.76126003e+00],
       [-2.54954600e+00,  2.43165708e+00],
       [-2.05594420e+00,  1.93019283e+00],
       [-3.16348743e+00,  3.07995963e+00],
       [-5.16400385e+00,  5.09361458e+00],
       [ 3.17241359e+00, -3.14490819e+00],
       [-4.13928843e+00,  3.99422431e+00],
       [ 1.74648154e+00, -1.85381317e+00],
       [ 3.02561808e+00, -3.20180249e+00],
       [ 3.59282565e+00, -3.67541337e+00],
       [ 9.89106655e-01, -1.08313537e+00],
       [-1.92848885e+00,  1.89781308e+00],
       [-3.59159493e+00,  3.35458469e+00],
       [-2.38872218e+00,  2.30616879e+00],
       [-2.90486073e+00,  2.74322796e+00],
       [-2.73772597e+00,  2.76172662e+00],
       [-3.45451880e+00,  3.31975412e+00],
       [-4.67230654e+00,  4.53815269e+00],
       [ 4.23241282e+00, -4.34206104e+00],
       [ 2.99258256e+00, -3.01362944e+00],
       [-3.76422024e+00,  3.62216449e+00],
       [ 1.76460159e+00, 

In [80]:
pred_test

PredictionOutput(predictions=array([[ 2.67072082e+00, -2.71354985e+00],
       [ 2.64563489e+00, -2.76126003e+00],
       [-2.54954600e+00,  2.43165708e+00],
       [-2.05594420e+00,  1.93019283e+00],
       [-3.16348743e+00,  3.07995963e+00],
       [-5.16400385e+00,  5.09361458e+00],
       [ 3.17241359e+00, -3.14490819e+00],
       [-4.13928843e+00,  3.99422431e+00],
       [ 1.74648154e+00, -1.85381317e+00],
       [ 3.02561808e+00, -3.20180249e+00],
       [ 3.59282565e+00, -3.67541337e+00],
       [ 9.89106655e-01, -1.08313537e+00],
       [-1.92848885e+00,  1.89781308e+00],
       [-3.59159493e+00,  3.35458469e+00],
       [-2.38872218e+00,  2.30616879e+00],
       [-2.90486073e+00,  2.74322796e+00],
       [-2.73772597e+00,  2.76172662e+00],
       [-3.45451880e+00,  3.31975412e+00],
       [-4.67230654e+00,  4.53815269e+00],
       [ 4.23241282e+00, -4.34206104e+00],
       [ 2.99258256e+00, -3.01362944e+00],
       [-3.76422024e+00,  3.62216449e+00],
       [ 1.76460159e+00, 

In [ ]:
pred_test = trainer.predict(dataset["test"])
pred_test

In [90]:
dataset["test"].features["idx"]

Value(dtype='int32', id=None)

In [91]:
dataset["test"]

Dataset({
    features: ['sentence', 'labels', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 1821
})

In [35]:
trainer.evaluate()

{'eval_loss': 0.3184843063354492,
 'eval_acc': 0.9025229357798165,
 'eval_runtime': 1.8486,
 'eval_samples_per_second': 471.717,
 'eval_steps_per_second': 15.147,
 'epoch': 6.0}

In [45]:
out_v = trainer.predict(dataset["validation"])

In [46]:
out_v

PredictionOutput(predictions=array([[-5.3419614 ,  5.3146486 ],
       [ 0.349477  , -0.4302054 ],
       [-3.788201  ,  3.7721741 ],
       ...,
       [ 0.87927294, -1.1090304 ],
       [ 3.084203  , -3.1437092 ],
       [-3.7187893 ,  3.5611577 ]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0

In [49]:
np.set_printoptions(threshold=sys.maxsize)

In [50]:
out = trainer.predict(dataset["test"])

In [51]:
out.label_ids

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

In [23]:
trainer.predict(dataset["validation"])

PredictionOutput(predictions=array([[-5.3419614 ,  5.3146486 ],
       [ 0.349477  , -0.4302054 ],
       [-3.788201  ,  3.7721741 ],
       ...,
       [ 0.87927294, -1.1090304 ],
       [ 3.084203  , -3.1437092 ],
       [-3.7187893 ,  3.5611577 ]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0

In [18]:
model

DistilBertAdapterModel(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlockWithAdapters(
          (attention): MultiHeadSelfAttentionWithAdapters(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): LoRALinear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (k_lin): LoRALinear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (v_lin): LoRALinear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (out_lin): Linear(

We can put our trained model into a _Transformers_ pipeline to be able to make new predictions conveniently:

In [10]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

The model 'DistilBertAdapterModel' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceClassification

RuntimeError: Placeholder storage has not been allocated on MPS device!

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [16]:
model.save_adapter("./final_adapter", "sst2")

!ls -lh final_adapter

total 8160
-rw-r--r--  1 mohamadh  staff   1.0K Apr 12 23:59 adapter_config.json
-rw-r--r--  1 mohamadh  staff   449B Apr 12 23:59 head_config.json
-rw-r--r--  1 mohamadh  staff   1.7M Apr 12 23:59 pytorch_adapter.bin
-rw-r--r--  1 mohamadh  staff   2.3M Apr 12 23:59 pytorch_model_head.bin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
model

**Share your work!**

The final step after successful training is to share our adapter with the world!
_Adapters_ seamlessly integrates with the [Hugging Face Model Hub](https://huggingface.co/models), so you can publish your trained adapter with a single method call:

**Important:** Make sure you're properly authenticated with your Hugging Face account before running this method. You can log in by running `huggingface-cli login` on your terminal.

In [ ]:
model.push_adapter_to_hub(
    "my-awesome-adapter",
    "rotten_tomatoes",
    adapterhub_tag="sentiment/rotten_tomatoes",
    datasets_tag="rotten_tomatoes"
)

This will create a repository _my-awesome-adapter_ under your username, generate a default adapter card as README.md and upload the adapter named `rotten_tomatoes` together with the adapter card to the new repository. Passing `adapterhub_tag` is required to make sure your adapter is features on [adapterhub.ml/explore](https://adapterhub.ml/explore), our Hub page. [Learn more](https://docs.adapterhub.ml/huggingface_hub.html).

➡️ Continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapters/blob/main/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.